In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Clustering process for policies

## 1. Import the relevant packages

In [2]:
import numpy as np
import pandas as pd
import random
import os

from process import extract_items
import clusterize

2025-07-28 10:10:45.400837: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-28 10:10:45.504360: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-28 10:10:45.626212: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-28 10:10:45.740999: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-28 10:10:45.741761: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-28 10:10:45.924514: I tensorflow/core/platform/cpu_feature_guard.cc:

## 2. Prepare data for clustering

In [3]:
domain = 'C'
input_path = f"../data/results_{domain}.csv"

In [4]:
# Import data
df = pd.read_csv(input_path)
df

,GEOGRAPHIC,ITEM,FACTOR,SECTOR,abstract,doi,openalex_id
0,offshore locations,Gas-to-Liquids (GTL) Floating Production Stora...,"{""development of offshore oil & gas fields"": {...","[""C"", ""D""]",Growing global energy demands have led us to d...,https://doi.org/10.4043/26098-ms,W2333443813
1,offshore locations,clean fuel and the chemical feedstock from the...,"{""profitability in the markets"": {""CORRELATION...","[""C""]",Growing global energy demands have led us to d...,https://doi.org/10.4043/26098-ms,W2333443813
2,"Shandong, eastern coastal China",readjusting industrial structure,"{""water shortage"": {""CORRELATION"": ""decreasing""}}","[""C""]",Water scarcity is one of the most difficult is...,https://doi.org/10.1016/j.proenv.2012.01.085,W2044047682
3,NaN,free trade,"{""international economic justice"": {""CORRELATI...","[""C""]",Recent years have seen a growing number of act...,NaN,W3124965391
4,NaN,limited flows of foreign assistance,"{""international economic justice"": {""CORRELATI...","[""C""]",Recent years have seen a growing number of act...,NaN,W3124965391
...,...,...,...,...,...,...,...
175966,NaN,design points and optimization techniques,"{""structural reliability of FPV systems"": {""CO...","[""C""]",As the global demand for energy continues to i...,https://doi.org/10.1063/5.0159394,W4387088031
175967,Croatian Electrical Power System,demand side management inside smart power grids,"{""balancing and regulation of energy productio...","[""C""]",The central purpose of this paper is to introd...,NaN,W2952907218
175968,Croatian Electrical Power System,renewable energy sources,"{""balancing and regulation of energy productio...","[""C""]",The central purpose of this paper is to introd...,NaN,W2952907218
175969,Croatian Electrical Power System,smart metering and energy box,"{""demand management programs"": {""CORRELATION"":...","[""C""]",The central purpose of this paper is to introd...,NaN,W2952907218


In [5]:
# Apply the extraction
extracted_df = extract_items(df)
extracted_df

"leaching recovery of REEs"
"leaching kinetics"
"leaching reaction"
"REE concentration"
"rare earth oxalate grade"
"contamination removal"
"cost reduction"
"leaching performance"
"oxalic acid consumption"
"coal recovery"
"REM recovery"
"industrial process heating demands"
"industrial process heating demands"


,index,ITEM,FACTOR,doi
0,0,Gas-to-Liquids (GTL) Floating Production Stora...,development of offshore oil & gas fields,https://doi.org/10.4043/26098-ms
1,0,Gas-to-Liquids (GTL) Floating Production Stora...,economic transportation,https://doi.org/10.4043/26098-ms
2,0,Gas-to-Liquids (GTL) Floating Production Stora...,infrastructure,https://doi.org/10.4043/26098-ms
3,0,Gas-to-Liquids (GTL) Floating Production Stora...,GTL conversion technology,https://doi.org/10.4043/26098-ms
4,1,clean fuel and the chemical feedstock from the...,profitability in the markets,https://doi.org/10.4043/26098-ms
...,...,...,...,...
264973,175966,design points and optimization techniques,structural reliability of FPV systems,https://doi.org/10.1063/5.0159394
264974,175967,demand side management inside smart power grids,balancing and regulation of energy production ...,NaN
264975,175968,renewable energy sources,balancing and regulation of energy production ...,NaN
264976,175969,smart metering and energy box,demand management programs,NaN


## 3. Cluster with HDBSCAN

### a. ITEMS

In [6]:
preprocessed_items = extracted_df[['index', 'ITEM']].drop_duplicates(subset='index')['ITEM']
preprocessed_items

0         Gas-to-Liquids (GTL) Floating Production Stora...
4         clean fuel and the chemical feedstock from the...
5                          readjusting industrial structure
6                                                free trade
7                       limited flows of foreign assistance
                                ...                        
264973            design points and optimization techniques
264974      demand side management inside smart power grids
264975                             renewable energy sources
264976                        smart metering and energy box
264977                              demand respond programs
Name: ITEM, Length: 173950, dtype: object

In [7]:
if os.path.exists(f'../embeddings/corpus_embeddings_items_{domain}.npy'):
    print('loading embeddings from file')
    corpus_embeddings_items = np.load(f'../embeddings/corpus_embeddings_items_{domain}.npy')
else:
    corpus_list_items, corpus_embeddings_items = clusterize.get_embeddings(preprocessed_items)
    np.save(f'../embeddings/corpus_embeddings_items_{domain}', corpus_embeddings_items)

if os.path.exists(f'../embeddings/corpus_list_items_{domain}.npy'):
    print('loading corpus list from file')
    corpus_list_items = np.load(f'../embeddings/corpus_list_items_{domain}.npy')
else:
    corpus_list_items, corpus_embeddings_items = clusterize.get_embeddings(preprocessed_items)
    np.save(f'../embeddings/corpus_list_items_{domain}', corpus_list_items)

loading embeddings from file
loading corpus list from file


In [8]:
params = {
    'umap_n_neighbors': 30,
    'umap_min_dist': 0.1,
    'umap_n_components': 50,
    'hdbscan_min_cluster_size': 300,
    'hdbscan_min_samples': 10,
    'hdbscan_cluster_selection_epsilon': 0.1,
    'hdbscan_max_cluster_size': None
}

hdbscan_model, cluster_assignment, clustered_sentences, noise, reduced_embeddings_items = clusterize.clusterize(
                            corpus_list_items, corpus_embeddings_items,
                            params,
                            print_clusters=False)

/home/thomas/.pyenv/versions/lewagon-decision-science/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Number of clusters found: 19


In [13]:
results = {
    'items_total': len(cluster_assignment),
    'items_noise': len(noise),
    'noise_pct': round(len(noise)/len(cluster_assignment) * 100, 2)
}

print(f'total number of items: {results['items_total']}, \nitems classified as noise: {results['items_noise']}, \npercentage of noise: {results['noise_pct']}')

total number of items: 173950, 
items classified as noise: 10939, 
percentage of noise: 6.29


In [14]:
cluster_summary_df = clusterize.get_cluster_summary(clustered_sentences)
cluster_summary_df

,Cluster Number,Number of Sentences,Sample Sentences
0,1,24,wind generated energy; wind Energy; Wind Energ...
1,2,16,large-scale solar energy development (SED); So...
2,3,25,renewable energy sources proliferation; renewa...
3,4,16,energy efficiency objectives; energy-efficienc...
4,5,12,renewable energy characteristics; renewable en...
5,6,17,Wind power-to-gas concepts; wind and minipower...
6,7,85,BES (battery energy storage); Battery Energy S...
7,8,325,Enhanced Geothermal Systems; geothermal energy...
8,9,132,diesel power generators; diesel gen sets; dies...
9,10,41,synthetic natural gas; Natural gas consumption...


In [ ]:
noise_df = pd.DataFrame.from_dict({'samples': random.sample(noise, 100)})
noise_df

,samples
0,solar power
1,renewable energy use
2,Virtual Power Plant system
3,renewable energy consumption
4,hydropower
...,...
95,renewable energy usage
96,Sb containing solder
97,renewable energy resources (RERs)
98,carbon tax


In [16]:
clusterize.save_results(params, results, noise_df, cluster_summary_df, hdbscan_model, domain, 'items')

In [ ]:
## Re-clsutering of subdivision of clusters
### List of clusters to subdivide
clusters_to_subdivide_with_params = {
    # 18: {'min_cluster_size': 100, 'min_samples': 5, 'cluster_selection_epsilon': 0.02} ### pas mal, tenter avec 0.03 ?
    18: {'min_cluster_size': 100, 'min_samples': 5, 'cluster_selection_epsilon': 0.025}
}

# Subdivide selected clusters and create a new dataframe
new_cluster_df, new_noise, cluster_assignment_to_change_indices, new_cluster_assignment = clusterize.subdivide_clusters(
    clustered_sentences, cluster_assignment, reduced_embeddings_items, preprocessed_items, clusters_to_subdivide_with_params
)

# Display the new dataframe
new_cluster_df



######### TODO améliorer le rendu du tableau de subclustering pour checker les gros clusters !! comme tableau plus haut

reclustering of cluster 18 adding 4223 items to noise


,cluster_id,sentences
0,1,"[wind energy, wind energy, wind energy, 100% w..."
1,2,"[solar energy, solar energy, solar energy, sol..."
2,3,"[renewable energy sources, renewable energy so..."
3,4,"[energy efficiency, energy efficiency potentia..."
4,5,"[renewable energy, renewable energy, renewable..."
...,...,...
64,18-46,[replacing alternative fuels with fossil fuels...
65,18-47,"[uranium exploration, production of uranium co..."
66,18-48,[sustainable electricity supply chain mathemat...
67,18-49,[Gas-to-Liquids (GTL) Floating Production Stor...


In [38]:
print(f'{new_cluster_df.shape[0] - cluster_summary_df.shape[0] + len(clusters_to_subdivide_with_params.keys())} new clusters were created out of {len(clusters_to_subdivide_with_params.keys())}')
print(f'{len(new_noise)} items from the previous clusters were classified as noise')

51 new clusters were created out of 1
4223 items from the previous clusters were classified as noise


In [55]:
recluster_results = {
    'items_total': len(cluster_assignment),
    'items_noise': len(noise) + len(new_noise),
    'noise_pct': round((len(noise) + len(new_noise))/len(cluster_assignment) * 100, 2)
}

pd.DataFrame.from_dict(recluster_results, orient='index', columns=['valeur']).to_csv(f'../outputs/results_recluster_{domain}_items.csv')

print(f'total number of items: {recluster_results['items_total']}, \nitems classified as noise: {recluster_results['items_noise']}, \npercentage of noise: {recluster_results['noise_pct']}')

total number of items: 173950, 
items classified as noise: 15162, 
percentage of noise: 8.72


In [39]:
new_noise[:10]

['installed wind capacity',
 'bioplastics production',
 'renewable energy integration',
 'square FA',
 'distributed photo-voltaic (PV) systems',
 'innovative co-generator',
 'multi-carrier energy grids',
 'fossil energy carriers',
 'intermediate temperature steam electrolyser (ITSE)',
 'renewable power generation']

In [ ]:
new_noise_df = pd.DataFrame({'noise_samples': random.sample(new_noise, min(100, len(new_noise)))})

new_noise_df.to_csv(f'../outputs/noise_sample_recluster_{domain}_items.csv', index=False)

new_noise_df

,noise_samples
0,reduced energy intensity
1,gas-fired cogeneration (CHP)
2,modern engines (Euro-III and Euro-IV compliant)
3,gross regional product
4,decentralized generation units
...,...
95,carbon emissions trading system (ETS)
96,mergers and acquisitions
97,integration of renewable energy sources (RES)
98,industrial structure


In [47]:
sentences_df = pd.DataFrame(preprocessed_items.copy())

doi_col = extracted_df[['index', 'doi']].drop_duplicates(subset='index')['doi']

cluster_assignment_with_correct_index = [i + 1 if i != -1 else i for i in cluster_assignment]

final_df = clusterize.create_final_df(sentences_df, doi_col, cluster_assignment_with_correct_index, new_cluster_assignment, cluster_assignment_to_change_indices)
final_df

,ITEM,cluster_id,recluster_id,doi
0,Gas-to-Liquids (GTL) Floating Production Stora...,18,18_49,https://doi.org/10.4043/26098-ms
4,clean fuel and the chemical feedstock from the...,18,18_49,https://doi.org/10.4043/26098-ms
5,readjusting industrial structure,18,18_49,https://doi.org/10.1016/j.proenv.2012.01.085
6,free trade,18,18_49,NaN
7,limited flows of foreign assistance,18,18_49,NaN
...,...,...,...,...
264973,design points and optimization techniques,18,18_49,https://doi.org/10.1063/5.0159394
264974,demand side management inside smart power grids,18,18_49,NaN
264975,renewable energy sources,3,3,NaN
264976,smart metering and energy box,11,11,NaN


In [48]:
final_df.to_csv(f'../outputs/clusters_recluster_hdbscan_{domain}_items.csv', index=False)

In [49]:
recluster_params = pd.DataFrame.from_dict(clusters_to_subdivide_with_params, orient='index')
recluster_params.index.name = 'cluster_id'

recluster_params.to_csv(f'../outputs/params_recluster_{domain}_items.csv')

recluster_params

,min_cluster_size,min_samples,cluster_selection_epsilon
cluster_id,,,
18,100,5,0.025


In [51]:
def suppress_and_merge_clusters(cluster_df, clusters_to_suppress, clusters_to_merge):
    """
    Suppress and merge clusters in a DataFrame.

    Parameters:
    - cluster_df: DataFrame containing cluster information.
    - clusters_to_suppress: List of cluster IDs to suppress.
    - clusters_to_merge: Dictionary where keys are clusters to keep, and values are lists of clusters to merge into them.

    Returns:
    - Updated DataFrame with suppressed and merged clusters.
    """
    # Suppress clusters
    suppressed_df = cluster_df[~cluster_df["cluster_id"].isin(clusters_to_suppress)]

    # Merge clusters
    for target_cluster, clusters_to_merge_into in clusters_to_merge.items():
        # Find the sentences for all clusters to merge
        sentences_to_merge = []
        for merge_cluster in clusters_to_merge_into:
            merge_rows = suppressed_df[suppressed_df["cluster_id"] == merge_cluster]
            if not merge_rows.empty:
                sentences_to_merge.extend(merge_rows.iloc[0]["sentences"])

        # Append sentences to the target cluster
        target_row = suppressed_df[suppressed_df["cluster_id"] == target_cluster]
        if not target_row.empty:
            target_row_index = target_row.index[0]
            suppressed_df.at[target_row_index, "sentences"] = (
                suppressed_df.at[target_row_index, "sentences"] + sentences_to_merge
            )

        # Remove merged clusters
        suppressed_df = suppressed_df[~suppressed_df["cluster_id"].isin(clusters_to_merge_into)]

    # Reset the index for a clean DataFrame
    suppressed_df.reset_index(drop=True, inplace=True)
    return suppressed_df


# Define clusters to suppress
clusters_to_suppress = []

# Define clusters to merge
clusters_to_merge = {
    '18_13': ['18_14'],
    '18_11': ['18_30']
}

# Apply suppression and merging
cleaned_cluster_df = suppress_and_merge_clusters(final_df, clusters_to_suppress, clusters_to_merge)

cleaned_cluster_df.to_csv(f'../outputs/clusters_recluster_hdbscan_{domain}_items.csv', index=False)

# Display the updated DataFrame
cleaned_cluster_df

,ITEM,cluster_id,recluster_id,doi
0,Gas-to-Liquids (GTL) Floating Production Stora...,18,18_49,https://doi.org/10.4043/26098-ms
1,clean fuel and the chemical feedstock from the...,18,18_49,https://doi.org/10.4043/26098-ms
2,readjusting industrial structure,18,18_49,https://doi.org/10.1016/j.proenv.2012.01.085
3,free trade,18,18_49,NaN
4,limited flows of foreign assistance,18,18_49,NaN
...,...,...,...,...
173945,design points and optimization techniques,18,18_49,https://doi.org/10.1063/5.0159394
173946,demand side management inside smart power grids,18,18_49,NaN
173947,renewable energy sources,3,3,NaN
173948,smart metering and energy box,11,11,NaN


### c. FACTORS

In [20]:
preprocessed_factors = extracted_df['FACTOR']
preprocessed_factors

0        awareness about the benefits of physical activity
1         physical activity (PA) in the general population
2                      PA barriers that AD caregivers face
3                                    trip generation rates
4                            intent to purchase activewear
                               ...                        
10376                air quality information accessibility
10377                            mobile telephone coverage
10378                                              obesity
10379                           biogas technology adoption
10380                                       social support
Name: FACTOR, Length: 10381, dtype: object

In [21]:
corpus_list_factors, corpus_embeddings_factors = clusterize.get_embeddings(preprocessed_factors)

In [22]:
params = {
    'umap_n_neighbors': 10,
    'umap_min_dist': 0.1,
    'umap_n_components': 60,
    'hdbscan_min_cluster_size': 40,
    'hdbscan_min_samples': 5,
    'hdbscan_cluster_selection_epsilon': 0.2,
    'hdbscan_max_cluster_size': None #300
}

hdbscan_model, cluster_assignment, clustered_sentences, noise, reduced_embeddings_factors = clusterize.clusterize(
                            corpus_list_factors, corpus_embeddings_factors,
                            params,
                            print_clusters=False)

/home/thomas/.pyenv/versions/lewagon-decision-science/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Number of clusters found: 21


In [23]:
results = {
    'factors_total': len(cluster_assignment),
    'factors_noise': len(noise),
    'noise_pct': round(len(noise)/len(cluster_assignment) * 100, 2)
}

print(f'total number of factors: {results['factors_total']}, \nfactors classified as noise: {results['factors_noise']}, \npercentage of noise: {results['noise_pct']}')

total number of factors: 10381, 
factors classified as noise: 1000, 
percentage of noise: 9.63


In [24]:
cluster_summary_df = clusterize.get_cluster_summary(clustered_sentences)
cluster_summary_df

,Cluster Number,Number of Sentences,Sample Sentences
0,1,7,perceptions of self-worth; affective well-bein...
1,2,3,free expression; freedom of expression; right ...
2,3,6,customer performance evaluation; customer moti...
3,4,6,sedentary life; sedentary lifestyle; carbon em...
4,5,12,carbon dioxide emissions; CO2 emissions; CO2 e...
5,6,3,sleep outcomes; subjective well-being (SWB); s...
6,7,2,well-being at work; well-being
7,8,11,carbon footprint; environmental footprint; ene...
8,9,14,global greenhouse gas emissions; embodied gree...
9,10,19,pain intensity; stress assessment; acculturati...


In [ ]:
noise_df = pd.DataFrame.from_dict({'samples': random.sample(noise, 100)})
noise_df

,samples
0,brown power consumption of transport networks ...
1,urban-rural income gap
2,total energy consumption of the BSs
3,brown power consumption of transport networks ...
4,EV charging process
...,...
95,e-Loyalty
96,e-Loyalty
97,cost savings
98,health knowledge


In [26]:
clusterize.save_results(params, results, noise_df, cluster_summary_df, hdbscan_model, domain, 'factors')

In [27]:
clusters_to_subdivide_with_params = {
    4: {'min_cluster_size': 10, 'min_samples': 5, 'cluster_selection_epsilon': 0.1},
    #15: {'min_cluster_size': 8, 'min_samples': 5, 'cluster_selection_epsilon': 0.0},
    20: {'min_cluster_size': 10, 'min_samples': 5, 'cluster_selection_epsilon': 0.1}
}

# Subdivide selected clusters and create a new dataframe
new_cluster_df, new_noise, cluster_assignment_to_change_indices, new_cluster_assignment = clusterize.subdivide_clusters(
    clustered_sentences, cluster_assignment, reduced_embeddings_factors, preprocessed_factors, clusters_to_subdivide_with_params
)

# Display the new dataframe
new_cluster_df

no noise generated by cluster 4
reclustering of cluster 20 adding 33 items to noise


,cluster_id,sentences
0,1,"[subjective well-being, subjective well-being,..."
1,2,"[freedom of expression, freedom of expression,..."
2,3,"[customer satisfaction, customer satisfaction,..."
3,5,"[CO2 emissions, CO2 emissions, carbon dioxide ..."
4,6,"[sleep quality, sleep quality, sleep quality, ..."
5,7,"[well-being, well-being, well-being, well-bein..."
6,8,"[environmental footprint, carbon footprint, ca..."
7,9,"[embodied greenhouse gas emissions, embodied g..."
8,10,"[stress management, stress resistance, stress ..."
9,11,"[Impulse Buying, Impulse Buying, Impulse Buyin..."


In [28]:
print(f'{new_cluster_df.shape[0] - cluster_summary_df.shape[0] + len(clusters_to_subdivide_with_params.keys())} new clusters were created out of {len(clusters_to_subdivide_with_params.keys())}')
print(f'{len(new_noise)} items from the previous clusters were classified as noise')

36 new clusters were created out of 2
33 items from the previous clusters were classified as noise


In [32]:
sentences_df = pd.DataFrame(preprocessed_factors.copy())

doi_col = extracted_df['doi']

cluster_assignment_with_correct_index = [i + 1 if i != -1 else i for i in cluster_assignment]

final_df = clusterize.create_final_df(sentences_df, doi_col, cluster_assignment_with_correct_index, new_cluster_assignment, cluster_assignment_to_change_indices)
final_df.to_csv(f'../outputs/clusters_recluster_hdbscan_{domain}_factors.csv', index=False)

final_df

,FACTOR,cluster,cluster_2,doi
0,awareness about the benefits of physical activity,20,20_16,https://doi.org/10.22038/jmrh.2014.2918
1,physical activity (PA) in the general population,20,20_16,https://doi.org/10.48550/arxiv.1908.09984
2,PA barriers that AD caregivers face,20,20_32,https://doi.org/10.48550/arxiv.1908.09984
3,trip generation rates,20,20_32,https://doi.org/10.3141/1682-09
4,intent to purchase activewear,20,20_32,https://doi.org/10.1080/17543266.2018.1477995
...,...,...,...,...
10376,air quality information accessibility,20,20_32,https://doi.org/10.51967/tanesa.v24i2.2946
10377,mobile telephone coverage,20,20_32,NaN
10378,obesity,20,20_32,NaN
10379,biogas technology adoption,-1,-1,https://doi.org/10.3390/ijerph17072311


In [ ]:
new_noise_df = pd.DataFrame({'noise_samples': random.sample(new_noise, min(100, len(new_noise)))})

new_noise_df.to_csv(f'../outputs/noise_sample_recluster_{domain}_factors.csv', index=False)

new_noise_df

,noise_samples
0,real impacts on human lives
1,real impacts on human lives
2,harmonious passion
3,job prospects
4,job prospects
5,real impacts on human lives
6,impact on disabled sex workers
7,social justice outcomes for sex workers
8,employment prospects
9,job prospects


In [31]:
recluster_params = pd.DataFrame.from_dict(clusters_to_subdivide_with_params, orient='index')
recluster_params.index.name = 'cluster_id'

recluster_params.to_csv(f'../outputs/params_recluster_{domain}_factors.csv')

recluster_params

,min_cluster_size,min_samples,cluster_selection_epsilon
cluster_id,,,
4,10,5,0.1
20,10,5,0.1
